In [21]:
import os
import sys

proj_root = os.path.abspath(os.getcwd())
if proj_root not in sys.path:
    sys.path.insert(0, proj_root)

In [22]:
# Zelle 2: HotelManager importieren und Instanz erstellen

from business_logic.hotel_manager import HotelManager
from data_access.hotel_data_access import HotelDataAccess
from data_access.room_data_access import RoomDataAccess
from data_access.booking_data_access import BookingDataAccess
from business_logic.room_manager import RoomManager
from data_access.facility_data_access import FacilityDataAccess
from data_access.address_data_access import AddressDataAccess
from business_logic.booking_manager import BookingManager
from business_logic.invoice_manager import InvoiceManager
from data_access.invoice_data_access import InvoiceDataAccess

import model
from model import hotel

# Pfad zur SQLite-Datenbank
db_path = os.path.join(proj_root, "database", "hotel_reservation_sample.db")

# DataAccess-Objekte erzeugen
hotel_dao = HotelDataAccess(db_path)
room_dao = RoomDataAccess(db_path)
booking_dao = BookingDataAccess(db_path)
facility_dao = FacilityDataAccess(db_path)
address_da = AddressDataAccess(db_path)
invoice_dao = InvoiceDataAccess(db_path)

# HotelManager mit dem Datenbankpfad erzeugen
manager = HotelManager(hotel_data_access=hotel_dao, room_data_access=room_dao)
room_manager = RoomManager(room_data_access=room_dao, facility_data_access=facility_dao)
booking_manager = BookingManager(booking_da=booking_dao, room_manager=room_manager)
invoice_manager = InvoiceManager(invoice_data_access=invoice_dao, booking_data_access=booking_dao)


1. Als Gast möchte ich die verfügbaren Hotels durchsuchen, damit
ich dasjenige auswählen kann, welches meinen Wünschen
entspricht. Wünsche sind:


1.1. Ich möchte alle Hotels in einer Stadt durchsuchen,
damit ich das Hotel nach meinem bevorzugten Standort
(Stadt) auswählen kann.


In [3]:
# DAO auf die DB zeigen lassen
dao = HotelDataAccess("database/hotel_reservation_sample.db")
manager = HotelManager(dao)

# nach Stadt filtern im String
hotels_in_WunschStadt = manager.show_hotels_by_city("Olten")

if not hotels_in_WunschStadt:
    print("Keine Hotels in dieser Stadt gefunden.")
else:
    for h in hotels_in_WunschStadt:
        print(f"{h.name} – {h.address.street}, {h.address.zip_code}, {h.address.city}")

Keine Hotels in dieser Stadt gefunden.


1.2. Ich möchte alle Hotels in einer Stadt nach der
Anzahl der Sterne (z.B. mindestens 4 Sterne) durchsuchen.

In [4]:
dao = HotelDataAccess("database/hotel_reservation_sample.db")
manager = HotelManager(dao)

# Stadt und min_stars angeben
city = "Basel"
min_stars = 5

# 3) Methode aufrufen
result = manager.show_hotels_by_city_and_min_stars(city, min_stars)

# 4) Ausgabe, falls Treffer vorhanden
if result:
    print(f"Gefundene Hotels in '{city}' mit mindestens {min_stars} Sternen:")
    for h in result:
        print(f"  • {h.name} ({h.stars} Sterne) – {h.address.street}, {h.address.zip_code}")

Gefundene Hotels in 'Basel' mit mindestens 5 Sternen:
  • Les Trois Rois (5 Sterne) – Freiestrasse 10, 4051


1.3. Ich möchte alle Hotels in einer Stadt durchsuchen,
die Zimmer haben, die meiner Gästezahl entsprechen (nur 1
Zimmer pro Buchung)

In [5]:
# 1) DAO-Objekte erstellen
dao = HotelDataAccess("database/hotel_reservation_sample.db")
room_dao = RoomDataAccess("database/hotel_reservation_sample.db")

# 2) HotelManager mit beiden DAOs initialisieren
manager = HotelManager(dao, room_dao)

# 3) Stadt und Gästeanzahl angeben
city = "zürich"
guest_count = 1

# 4) Methode aufrufen
result = manager.find_hotels_by_city_and_guests(city, guest_count)

# 5) Ausgabe, falls Treffer vorhanden
if result:
    print(f"Gefundene Hotels in '{city}' mit Zimmern für mindestens {guest_count} Gäste:")
    for h in result:
        print(f"  • {h.name} – {h.address.street}, {h.address.zip_code}")
else:
    print(f"Keine Hotels in '{city}' mit Zimmern für mindestens {guest_count} Gäste gefunden.")

Gefundene Hotels in 'zürich' mit Zimmern für mindestens 1 Gäste:
  • Hotel Baur au Lac – Bahnhofstrasse 1, 8001


1.4. Ich möchte alle Hotels in einer Stadt durchsuchen,
die während meines Aufenthaltes ("von" (check_in_date)
und "bis" (check_out_date)) Zimmer zur Verfügung haben,
damit ich nur relevante Ergebnisse sehe.

In [6]:
from datetime import date

# Abfrageparameter definieren
city = "Genève"                 ## Beispiel Buchung in DB in Genève ist von 2025, 8, 20 bis  2025, 8, 22
check_in = date(2025, 8, 1)
check_out = date(2025, 8, 17)

# Verfügbare Hotels abrufen
verfuegbare_hotels = manager.find_available_hotels_by_city_and_dates(city, check_in, check_out, booking_dao)

# Ausgabe der Ergebnisse
if verfuegbare_hotels:
    print(f"Verfügbare Hotels in '{city}' vom {check_in} bis {check_out}:")
    for h in verfuegbare_hotels:
        print(f"{h.name} – {h.address.street}, {h.address.zip_code}")
else:
    print(f"Keine verfügbaren Hotels in '{city}' vom {check_in} bis {check_out}.")

Verfügbare Hotels in 'Genève' vom 2025-08-01 bis 2025-08-17:
Four Seasons Hôtel des Bergues – Rue du Rhône 42, 1204


1.5. Ich möchte Wünsche kombinieren können, z.B. die
verfügbaren Zimmer zusammen mit meiner Gästezahl und der
mindest Anzahl Sterne.


In [7]:
from datetime import datetime

# Parameter definieren --> Hotel in Genf hat 5 Sterne, Room für max 4 Leute uund booking von 2025-8-20 - 2025-8-22
stadt = "Genève"
check_in = datetime(2025, 8, 23)
check_out = datetime(2025, 8, 24)
min_sterne = 4
gaesteanzahl = 2

# Methode aufrufen
verfuegbare_hotels = manager.find_hotels_by_criteria(
    city=stadt,
    check_in_date=check_in,
    check_out_date=check_out,
    min_stars=min_sterne,
    guest_count=gaesteanzahl,
    booking_da=booking_dao
)

# Ausgabe
if verfuegbare_hotels:
    print(f"Verfügbare Hotels in '{stadt}' vom {check_in.date()} bis {check_out.date()} (mind. {min_sterne} Sterne, für {gaesteanzahl} Gäste):")
    for hotel in verfuegbare_hotels:
        print(f"  • {hotel.name} – {hotel.address.street}, {hotel.address.zip_code} {hotel.address.city}")
else:
    print(f"Keine passenden Hotels in '{stadt}' für die angegebenen Kriterien gefunden.")

Verfügbare Hotels in 'Genève' vom 2025-08-23 bis 2025-08-24 (mind. 4 Sterne, für 2 Gäste):
  • Four Seasons Hôtel des Bergues – Rue du Rhône 42, 1204 Genève


1.6. Ich möchte die folgenden Informationen pro Hotel
sehen: Name, Adresse, Anzahl der Sterne.

In [8]:
infos = manager.show_all_hotel_infos()

print("Verfügbare Hotels:")
for eintrag in infos:
    print(" •", eintrag)


Verfügbare Hotels:
 • Hotel Baur au Lac – Bahnhofstrasse 1, 8001 Zürich (5 Sterne)
 • Four Seasons Hôtel des Bergues – Rue du Rhône 42, 1204 Genève (5 Sterne)
 • Grand Hotel National – Gassstrassse 7, 4657 Dulliken (5 Sterne)
 • Bellevue Palace – Marktgasse 59, 3011 Bern (5 Sterne)
 • Les Trois Rois – Freiestrasse 10, 4051 Basel (5 Sterne)
 • bronzehand – Gassstrassse 7, 4657 Dulliken (5 Sterne)


2. Als Gast möchte ich Details zu verschiedenen Zimmertypen
(Single, Double, Suite usw.), die in einem Hotel verfügbar
sind, sehen, einschliesslich der maximalen Anzahl von Gästen
für dieses Zimmer, Beschreibung, Preis und Ausstattung, um eine
fundierte Entscheidung zu treffen.

2.1. Ich möchte die folgenden Informationen pro Zimmer
sehen: Zimmertyp, max. Anzahl der Gäste, Beschreibung,
Ausstattung, Preis pro Nacht und Gesamtpreis.


In [9]:
from datetime import date

# Optional: Check-in und Check-out setzen (für Gesamtpreisberechnung)
check_in = date(2025, 7, 11)
check_out = date(2025, 7, 15)

# Zimmerdetails abrufen
room_infos = room_manager.show_room_details()

# Ausgabe
for room in room_infos:
    ausstattung = ", ".join([f.facility_name for f in room.facilities])

    print(f"Zimmer {room.room_number} – {room.room_type.description}")
    print(f"  Max. Gäste     : {room.room_type.max_guests}")
    print(f"  Ausstattung    : {ausstattung if ausstattung else 'Keine'}")

    # Preis pro Nacht
    print(f"  Preis pro Nacht: {room.price_per_night:.2f} CHF")

    # Optional: Gesamtpreis berechnen, wenn Check-in/Check-out gesetzt
    if check_in and check_out and check_out > check_in:
        nächte = (check_out - check_in).days
        gesamtpreis = room.price_per_night * nächte
        print(f"  Gesamtpreis    : {gesamtpreis:.2f} CHF für {nächte} Nächte")

    print("-" * 60)

Zimmer 101 – Single
  Max. Gäste     : 1
  Ausstattung    : WiFi, TV
  Preis pro Nacht: 250.00 CHF
  Gesamtpreis    : 1000.00 CHF für 4 Nächte
------------------------------------------------------------
Zimmer 102 – Double
  Max. Gäste     : 2
  Ausstattung    : WiFi
  Preis pro Nacht: 400.00 CHF
  Gesamtpreis    : 1600.00 CHF für 4 Nächte
------------------------------------------------------------
Zimmer 201 – Suite
  Max. Gäste     : 4
  Ausstattung    : Air Conditioning
  Preis pro Nacht: 650.00 CHF
  Gesamtpreis    : 2600.00 CHF für 4 Nächte
------------------------------------------------------------
Zimmer 301 – Family Room
  Max. Gäste     : 5
  Ausstattung    : Mini Bar
  Preis pro Nacht: 900.00 CHF
  Gesamtpreis    : 3600.00 CHF für 4 Nächte
------------------------------------------------------------
Zimmer 401 – Penthouse
  Max. Gäste     : 6
  Ausstattung    : Keine
  Preis pro Nacht: 1500.00 CHF
  Gesamtpreis    : 6000.00 CHF für 4 Nächte
--------------------------------

2.2. Ich möchte nur die verfügbaren Zimmer sehen, sofern
ich meinen Aufenthalt (von – bis) spezifiziert habe.


In [10]:
from datetime import date

check_in = date(2025, 8, 20)
check_out = date(2025, 8, 22)

verfuegbare_zimmer = room_manager.find_available_rooms_by_dates(
    check_in_date=check_in,
    check_out_date=check_out,
    booking_dao=booking_dao
)

for room in verfuegbare_zimmer:
    ausstattung = ", ".join([f.facility_name for f in room.facilities])
    print(f"Zimmer {room.room_number} – {room.room_type.description}")
    print(f"  Hotel: {room.hotel.name}, Adresse: {room.hotel.address.street} {room.hotel.address.city}")
    print(f"  Max. Gäste     : {room.room_type.max_guests}")
    print(f"  Ausstattung    : {ausstattung if ausstattung else 'Keine'}")
    print(f"  Preis pro Nacht: {room.price_per_night:.2f} CHF")

    # Optional: Gesamtpreis berechnen
    if check_in and check_out and check_out > check_in:
        nächte = (check_out - check_in).days
        gesamtpreis = room.price_per_night * nächte
        print(f"  Gesamtpreis    : {gesamtpreis:.2f} CHF für {nächte} Nächte")

    print("-" * 60)

Zimmer 101 – Single
  Hotel: Hotel Baur au Lac, Adresse: Bahnhofstrasse 1 Zürich
  Max. Gäste     : 1
  Ausstattung    : WiFi, TV
  Preis pro Nacht: 250.00 CHF
  Gesamtpreis    : 500.00 CHF für 2 Nächte
------------------------------------------------------------
Zimmer 102 – Double
  Hotel: Hotel Baur au Lac, Adresse: Bahnhofstrasse 1 Zürich
  Max. Gäste     : 2
  Ausstattung    : WiFi
  Preis pro Nacht: 400.00 CHF
  Gesamtpreis    : 800.00 CHF für 2 Nächte
------------------------------------------------------------
Zimmer 301 – Family Room
  Hotel: Grand Hotel National, Adresse: Gassstrassse 7 Dulliken
  Max. Gäste     : 5
  Ausstattung    : Mini Bar
  Preis pro Nacht: 900.00 CHF
  Gesamtpreis    : 1800.00 CHF für 2 Nächte
------------------------------------------------------------
Zimmer 401 – Penthouse
  Hotel: Bellevue Palace, Adresse: Marktgasse 59 Bern
  Max. Gäste     : 6
  Ausstattung    : Keine
  Preis pro Nacht: 1500.00 CHF
  Gesamtpreis    : 3000.00 CHF für 2 Nächte
-----

3. Als Admin des Buchungssystems möchte ich die Möglichkeit haben,
Hotelinformationen zu pflegen, um aktuelle Informationen im
System zu haben.

3.1. Ich möchte neue Hotels zum System hinzufügen


In [11]:
from data_access.address_data_access import AddressDataAccess
from model import Address

# AddressDataAccess initialisieren
address_dao = AddressDataAccess(db_path)

# Adresse anlegen
adresse = Address(
    address_id=0,  # Platzhalter, wird von DB ersetzt
    street="Lindenweg 8",
    city="Däniken",
    zip_code=3012
)
# Hotel erstellen
try:
    neues_hotel = manager.create_new_hotel(
        name="Hotel Bernblick",
        stars=3,
        address=adresse,
        address_da=address_dao
    )
    print("Hotel erfolgreich erstellt:")
    print(f"Hotel-ID: {neues_hotel.hotel_id}")
    print(f"Address-ID: {neues_hotel.address.address_id}")
    print(f"{neues_hotel.name} – {neues_hotel.address.street}, {neues_hotel.address.zip_code} {neues_hotel.address.city}")
except Exception as e:
    print("Fehler beim Erstellen des Hotels:", e)

Hotel erfolgreich erstellt:
Hotel-ID: 17
Address-ID: 10
Hotel Bernblick – Lindenweg 8, 3012 Däniken


3.2. Ich möchte Hotels aus dem System entfernen

In [12]:
# Beispiel: Hotel mit ID 15 löschen
hotel_id_to_delete = 28

# Vorher: Liste aller Hotels anzeigen
print("Hotels vor dem Löschen:")
for h in manager.show_all_hotels_basic():
    print(f"Hotel-ID: {h.hotel_id} | Name: {h.name}")

# Hotel löschen
erfolgreich = manager.delete_hotel(hotel_id_to_delete)

# Rückmeldung
if erfolgreich:
    print(f"\nHotel mit ID {hotel_id_to_delete} wurde gelöscht.")
else:
    print(f"\nKein Hotel mit ID {hotel_id_to_delete} gefunden – nichts gelöscht.")

# Danach: Liste nochmals anzeigen
print("\nHotels nach dem Löschen:")
for h in manager.show_all_hotels_basic():
    print(f"Hotel-ID: {h.hotel_id} | Name: {h.name}")

Hotels vor dem Löschen:
Hotel-ID: 1 | Name: Hotel Baur au Lac
Hotel-ID: 2 | Name: Four Seasons Hôtel des Bergues
Hotel-ID: 3 | Name: Grand Hotel National
Hotel-ID: 4 | Name: Bellevue Palace
Hotel-ID: 5 | Name: Les Trois Rois
Hotel-ID: 16 | Name: bronzehand
Hotel-ID: 17 | Name: Hotel Bernblick

Kein Hotel mit ID 28 gefunden – nichts gelöscht.

Hotels nach dem Löschen:
Hotel-ID: 1 | Name: Hotel Baur au Lac
Hotel-ID: 2 | Name: Four Seasons Hôtel des Bergues
Hotel-ID: 3 | Name: Grand Hotel National
Hotel-ID: 4 | Name: Bellevue Palace
Hotel-ID: 5 | Name: Les Trois Rois
Hotel-ID: 16 | Name: bronzehand
Hotel-ID: 17 | Name: Hotel Bernblick


3.3. Ich möchte die Informationen bestimmter Hotels
aktualisieren, z. B. den Namen, die Sterne usw.


In [13]:
# Hotel-ID auswählen zum anpassen
hotel_id = 16

# Vorherige Daten anzeigen
original_hotel = next((h for h in hotel_dao.read_all_hotel() if h.hotel_id == hotel_id), None)

if not original_hotel:
    print(f"Hotel mit ID {hotel_id} wurde nicht gefunden.")
else:
    print("Vorher:")
    print(f"Name: {original_hotel.name}, Sterne: {original_hotel.stars}")
    print(f"Adresse: {original_hotel.address.street}, {original_hotel.address.zip_code} {original_hotel.address.city}")

    # Neue Werte definieren
    neuer_name = "bronzehand"
    neue_sterne = 5
    neue_adresse = model.Address(
        address_id=original_hotel.address.address_id,
        street="Gassstrassse 7",
        city="Dulliken",
        zip_code=4657
    )
    # Update durchführen
    ok = manager.update_hotel_and_address(
        hotel_id=hotel_id,
        name=neuer_name,
        stars=neue_sterne,
        address=neue_adresse,
        address_da=address_da
    )
    # Ergebnis anzeigen
    if ok:
        updated = next((h for h in hotel_dao.read_all_hotel() if h.hotel_id == hotel_id), None)
        print("\nNachher:")
        print(f"Name: {updated.name}, Sterne: {updated.stars}")
        print(f"Adresse: {updated.address.street}, {updated.address.zip_code} {updated.address.city}")
    else:
        print("Update fehlgeschlagen.")

Vorher:
Name: bronzehand, Sterne: 5
Adresse: Gassstrassse 7, 4657 Dulliken

Nachher:
Name: bronzehand, Sterne: 5
Adresse: Gassstrassse 7, 4657 Dulliken


4. Als Gast möchte ich ein Zimmer in einem bestimmten Hotel
buchen, um meinen Urlaub zu planen.


In [14]:
from datetime import date
from model import Guest, Room, RoomType, Hotel, Address

# Dummy-Daten (IDs müssen existieren!)
adresse = Address(address_id=1, street="Teststrasse 11", city="Olten", zip_code=4600)
hotel = Hotel(hotel_id=1, name="Hotel Test", stars=3, address=adresse)
room_type = RoomType(room_type_id=1, description="Standard", max_guests=2)
zimmer = Room(room_id=1, room_number="101", price_per_night=120.0, room_type=room_type, hotel=hotel)
gast = Guest(guest_id=1, firstname="Max", lastname="Muster", email="max@muster.ch")

check_in = date(2025, 12, 17)
check_out = date(2025, 12, 19)

# 1. Buchungen vor dem Einfügen anzeigen
print("Buchungen vor dem Einfügen:")
for b in booking_manager.show_bookings():
    print(f"Booking-ID: {b.booking_id}, Room-ID: {b.room_id}, Hotel-ID: {b.hotel_id}, Check-in: {b.check_in_date}, Check-out: {b.check_out_date}")

# 2. Verfügbarkeit prüfen und ggf. buchen
verfuegbare_zimmer = room_manager.find_available_rooms_by_dates(check_in, check_out, booking_dao)
neue_buchung = None

if any(r.room_id == zimmer.room_id for r in verfuegbare_zimmer):
    neue_buchung = booking_dao.create_booking(
        guest_id=gast,
        room_id=zimmer,
        check_in_date=check_in,
        check_out_date=check_out,
        is_cancelled=False,
        total_amount=zimmer.price_per_night * (check_out - check_in).days
    )
    print("\nNeue Buchung erfolgreich erstellt:")
    print(f"Booking-ID: {neue_buchung.booking_id}, Room-ID: {neue_buchung.room_id}, Zeitraum: {check_in} bis {check_out}")
else:
    print("\nZimmer ist im gewünschten Zeitraum nicht verfügbar.")

# 3. Buchungen nach dem Einfügen anzeigen
print("\nBuchungen nach dem Einfügen:")
for b in booking_manager.show_bookings():
    print(f"Booking-ID: {b.booking_id}, Room-ID: {b.room_id}, Hotel-ID: {b.hotel_id}, Check-in: {b.check_in_date}, Check-out: {b.check_out_date}")

Buchungen vor dem Einfügen:
Booking-ID: 1, Room-ID: 1, Hotel-ID: 1, Check-in: 2025-06-01, Check-out: 2025-06-05
Booking-ID: 2, Room-ID: 2, Hotel-ID: 1, Check-in: 2025-07-10, Check-out: 2025-07-15
Booking-ID: 3, Room-ID: 3, Hotel-ID: 2, Check-in: 2025-08-20, Check-out: 2025-08-22
Booking-ID: 4, Room-ID: 4, Hotel-ID: 3, Check-in: 2025-09-05, Check-out: 2025-09-10
Booking-ID: 5, Room-ID: 5, Hotel-ID: 4, Check-in: 2025-10-01, Check-out: 2025-10-07
Booking-ID: 6, Room-ID: 1, Hotel-ID: 1, Check-in: 2025-07-10, Check-out: 2025-07-15
Booking-ID: 7, Room-ID: 1, Hotel-ID: 1, Check-in: 2025-07-10, Check-out: 2025-07-15
Booking-ID: 8, Room-ID: 1, Hotel-ID: 1, Check-in: 2025-08-10, Check-out: 2025-08-15
Booking-ID: 9, Room-ID: 1, Hotel-ID: 1, Check-in: 2025-10-10, Check-out: 2025-10-15
Booking-ID: 10, Room-ID: 1, Hotel-ID: 1, Check-in: 2025-11-10, Check-out: 2025-11-15
Booking-ID: 11, Room-ID: 2, Hotel-ID: 1, Check-in: 2025-11-10, Check-out: 2025-11-15
Booking-ID: 12, Room-ID: 5, Hotel-ID: 4, Check

5. Als Gast möchte ich nach meinem Aufenthalt eine Rechnung
erhalten, damit ich einen Zahlungsnachweis habe.
Hint: Fügt einen Eintrag in der «Invoice» Tabelle hinzu.


In [15]:
try:
    booking_id = int(input("Booking-ID eingeben: "))
    invoice = invoice_manager.create_invoice_for_existing_booking(booking_id)

    print("Rechnung erfolgreich erstellt:")
    print(f"Rechnungs-ID: {invoice.invoice_id}")
    print(f"Zu Buchung:   {invoice.booking}")
    print(f"Betrag:       {invoice.total_amount} CHF")
    print(f"Rechnungsdatum: {invoice.issue_date}")
except Exception as e:
    print(f"Fehler beim Erstellen der Rechnung: {e}")

KeyboardInterrupt: Interrupted by user

6.Als Gast möchte ich meine Buchung stornieren, damit ich nicht
belastet werde, wenn ich das Zimmer nicht mehr benötige.
Hint: Sorgt für die entsprechende Invoice

In [25]:
# Booking-ID auswählen zum anpassen
booking_id = 5

# Vorherige Daten anzeigen
original_booking = next((b for b in booking_manager.show_bookings() if b.booking_id == booking_id), None)

if not original_booking:
    print(f"Booking mit ID {booking_id} wurde nicht gefunden.")
else:
    print("Vorher:")
    print(f"Guest: {original_booking.guest},")
    print(      f" Check-In: {original_booking.check_in_date}, Check-Out: {original_booking.check_out_date}")
    print(f"Storniert?: {original_booking.is_cancelled}, Gesamtbetrag {original_booking.total_amount}")
    #Ausschliessen, dass die Buchung vorher storniert war
if not original_booking.is_cancelled:
    # Neue Werte definieren
    cancelled = True
    # Update durchführen
    ok = booking_manager.update_booking(
        booking_id=booking_id,
        check_in_date=original_booking.check_in_date,
        check_out_date=original_booking.check_out_date,
        is_cancelled=cancelled,
        total_amount=original_booking.total_amount,
        guest_id=original_booking.guest
    )
    # Ergebnis anzeigen
    if ok:
        gelöscht = invoice_dao.delete_invoice_by_booking_id(booking_id)
        if gelöscht:
            print(f"Die zugehörige Rechnung zur Buchung {booking_id} wurde gelöscht")
        else:
            print(f"Es wurde keine Rechnung zur Buchung {booking_id} gefunden.")

        updated = next((b for b in booking_manager.show_bookings() if b.booking_id == booking_id), None)
        print("\nNachher:"),
        print(f"Guest-ID: {updated.guest}, Check-In: {updated.check_in_date}, Check-Out: {updated.check_out_date}")
        print(f"Storniert: {updated.is_cancelled}, {updated.total_amount} CHF")
    else:
        print("Update fehlgeschlagen.")
else: print("Buchung ist bereits storniert.")

Vorher:
Guest: 5,
 Check-In: 2025-10-01, Check-Out: 2025-10-07
Storniert?: 1, Gesamtbetrag 9000.0
Buchung ist bereits storniert.


7. Als Gast möchte ich eine dynamische Preisgestaltung auf der
Grundlage der Nachfrage sehen, damit ich ein Zimmer zum besten
Preis buchen kann.
Hint: Wendet in der Hochsaison höhere und in der Nebensaison
niedrigere Tarife an


In [19]:
from datetime import date

# beispielzimmer holen
rooms = room_manager.show_room_details()
room = rooms[2]  # Erstes Zimmer

# Zeitraum definieren
check_in = date(2025, 7, 10)
check_out = date(2025, 7, 15)

# Preisberechnung
base_price = room.price_per_night
num_nights = (check_out - check_in).days
total_base_price = base_price * num_nights
dynamic_price = room_manager.calculate_dynamic_price(room, check_in, check_out)

# Ausgabe
print(f"Zimmernummer: {room.room_number}")
print(f"Zeitraum: {check_in} bis {check_out} ({num_nights} Nächte)")
print(f"Basispreis pro Nacht: {base_price:.2f} CHF")
print(f"Gesamtpreis ohne Dynamik: {total_base_price:.2f} CHF")
print(f"Dynamischer Gesamtpreis: {dynamic_price:.2f} CHF")

Zimmernummer: 201
Zeitraum: 2025-07-10 bis 2025-07-15 (5 Nächte)
Basispreis pro Nacht: 650.00 CHF
Gesamtpreis ohne Dynamik: 3250.00 CHF
Dynamischer Gesamtpreis: 3900.00 CHF


X. Hilfsfunktion Bookings löschen.

In [ ]:
# Beispiel: Buchung mit ID 19 löschen
booking_id_to_delete = 6
#
# Vorher: Liste aller Buchungen anzeigen
print("Buchungen vor dem Löschen:")
for b in booking_manager.show_bookings():
    print(f"Booking-ID: {b.booking_id} | Room-ID: {b.room_id} | Hotel-ID: {b.hotel_id} | Zeitraum: {b.check_in_date} bis {b.check_out_date}")

# Buchung löschen
erfolgreich = booking_dao.delete_booking_by_id(booking_id_to_delete)

if erfolgreich:
    print(f"\nBuchung mit ID {booking_id_to_delete} wurde gelöscht.")
else:
    print(f"\nKeine Buchung mit ID {booking_id_to_delete} gefunden – nichts gelöscht.")

#Liste nochmals anzeigen
print("\nBuchungen nach dem Löschen:")
for b in booking_manager.show_bookings():
    print(f"Booking-ID: {b.booking_id} | Room-ID: {b.room_id} | Hotel-ID: {b.hotel_id} | Zeitraum: {b.check_in_date} bis {b.check_out_date}")